In [60]:
import PyPDF2
import requests
from urllib.request import Request, urlopen
import io
import pandas as pd
import re
import numpy as np

Colleges: <br>
AG: Agriculture and Life Sciences<br>
AR: Architecture<br>
GB: Bush School of Government and Public Service<br>
BA: Business<br>
DN: Dentistry<br>
ED: Education<br>
EN: Engineering<br>
GE: Geosciences<br>
LA: Liberal Arts<br>
MS: Military Science<br>
NU: Nursing<br>
PH: Public Health<br>
SC: Science<br>
VM: Vet Med<br>
Terms:<br>
1: Spring<br>
2: Summer<br>
3: Fall<br>
<br>
Data available from Fall 2017 - Spring 2022 as of Sept 9, 2022

In [16]:
colleges = ['AG', 'AR', 'GB', 'BA', 'DN','ED','EN','GE', 'LA',
            'MS','NU','PH','SC','VM']
terms = ['1','2','3'] 
years = ['2018','2019','2020','2021']

urls = []
#Only Fall available in 2017 
for college in colleges:
    urls.append('https://web-as.tamu.edu/GradeReports/PDFReports/20173/grd20173' + college + '.pdf')

#Get most of course data
for college in colleges:
    for year in years:
        for term in terms:
            urls.append('https://web-as.tamu.edu/GradeReports/PDFReports/'+ year + term + '/grd' + year + term + college + '.pdf')

#Only Spring available in 2022           
for college in colleges:
    if college == 'DN':
        continue
    urls.append('https://web-as.tamu.edu/GradeReports/PDFReports/20221/grd20221' + college + '.pdf')


In [36]:
rows = []
for URL in urls:
    print(URL)
    #Find file from URL
    file = urlopen(Request(URL)).read()
    mem_file = io.BytesIO(file)
    fileReader = PyPDF2.PdfFileReader(mem_file)
    
    #Find Course Name to parse the PDF
    course_list = set()
    page_count = fileReader.numPages
    for page_num in range(page_count):
        current_page = fileReader.getPage(page_num).extractText()
        courses = re.findall("[A-Z][A-Z][A-Z][A-Z]-[0-9]", current_page) 
        courses = set(courses)
        courses = list(courses)
        for elem in courses:
            course_list.add(elem)
            
    #Parse through PDF and create rows for data frame
    for page_num in range(page_count):
        current_page = fileReader.getPage(page_num).extractText().split()
        for num in range(len(current_page)):
            current_row = []
            
            #If the first 6 characters of the row is a valid course, create a row of data
            if current_page[num][:6] in course_list:
                instructor_name = ''
                try:
                    instructor_name = current_page[num+19] + ' ' + current_page[num+20]
                except:
                    instructor_name = current_page[num+19]
                current_row = current_page[num:num+19]
                current_row.append(instructor_name)
                
                #Add Year, Term, and College to record
                current_row.append(URL[48:52])
                current_row.append(URL[52])
                current_row.append(URL[62:64])
                #Add record to nested list
                rows.append(current_row)       

https://web-as.tamu.edu/GradeReports/PDFReports/20173/grd20173AG.pdf
{'FIVS-2', 'HORT-2', 'HORT-3', 'RENR-3', 'SCSC-2', 'ANSC-4', 'ENTO-4', 'BAEN-6', 'WFSC-3', 'POSC-3', 'ALEC-2', 'ESSM-2', 'AGSM-3', 'FSTC-6', 'BESC-4', 'WFSC-4', 'AGSC-4', 'RENR-2', 'RPTS-2', 'ALED-1', 'ALED-4', 'FSTC-2', 'ENTO-6', 'SCSC-6', 'AGCJ-2', 'ANSC-3', 'GENE-6', 'GENE-1', 'FIVS-1', 'AGEC-6', 'BICH-1', 'NUTR-4', 'AGCJ-3', 'HORT-4', 'FIVS-4', 'RPTS-3', 'NUTR-2', 'ANSC-1', 'POSC-4', 'ENTO-3', 'AGSC-3', 'ENTO-2', 'RPTS-4', 'WFSC-1', 'FIVS-3', 'SCSC-3', 'ESSM-4', 'AGSM-2', 'POSC-2', 'AGSM-1', 'AGEC-4', 'PLPA-6', 'GENE-3', 'ALED-2', 'POSC-6', 'BESC-2', 'RENR-4', 'AGSM-4', 'FSTC-3', 'NUTR-3', 'ALEC-4', 'ESSM-6', 'BICH-6', 'AGLS-1', 'AGEC-3', 'ANSC-6', 'AGCJ-1', 'SCSC-1', 'BICH-4', 'HORT-6', 'SCSC-4', 'ANSC-2', 'BAEN-3', 'GENE-4', 'MEPS-3', 'ESSM-1', 'AGCJ-4', 'BAEN-4', 'ALEC-6', 'ESSM-3', 'BICH-3', 'FSTC-4', 'BESC-3', 'AGEC-2', 'WFSC-6', 'ALEC-3', 'ALED-3', 'EEBL-6', 'PLPA-3', 'DASC-3', 'RPTS-6', 'AGEC-1', 'NUTR-6'}


https://web-as.tamu.edu/GradeReports/PDFReports/20191/grd20191AG.pdf
{'HORT-2', 'NFSC-6', 'HORT-3', 'NFSC-3', 'RENR-3', 'ANSC-4', 'ENTO-4', 'BAEN-6', 'WFSC-3', 'POSC-3', 'ALEC-2', 'ESSM-2', 'AGSM-3', 'BESC-4', 'WFSC-4', 'AGSC-4', 'RENR-2', 'RPTS-2', 'ALED-4', 'SCSC-6', 'ENTO-6', 'AGCJ-2', 'ANSC-3', 'GENE-6', 'FRSC-4', 'FIVS-1', 'AGEC-6', 'AGCJ-3', 'HORT-4', 'FIVS-4', 'RPTS-3', 'ANSC-1', 'POSC-4', 'ENTO-3', 'NFSC-2', 'AGSC-3', 'ENTO-2', 'NFSC-4', 'RPTS-4', 'SCSC-3', 'ESSM-4', 'AGSM-2', 'POSC-2', 'AGEC-4', 'PLPA-6', 'GENE-3', 'ALED-2', 'POSC-6', 'BESC-2', 'RENR-4', 'AGSM-4', 'ALEC-4', 'ESSM-6', 'BICH-6', 'AGLS-1', 'AGEC-3', 'ANSC-6', 'AGCJ-1', 'SCSC-1', 'BAEN-2', 'BICH-4', 'HORT-6', 'SCSC-4', 'ANSC-2', 'BAEN-3', 'MEPS-6', 'GENE-4', 'MEPS-3', 'ESSM-1', 'AGCJ-4', 'RENR-6', 'BAEN-4', 'ALEC-6', 'ESSM-3', 'BICH-3', 'FSTC-4', 'BESC-3', 'AGEC-2', 'WFSC-6', 'ALED-3', 'PLPA-3', 'DASC-3', 'RPTS-6', 'AGEC-1'}
https://web-as.tamu.edu/GradeReports/PDFReports/20192/grd20192AG.pdf
{'ESSM-4', 'FIVS-2', 

{'LDEV-6', 'LAND-4', 'PLAN-6', 'CARC-3', 'COSC-3', 'URSC-6', 'COSC-2', 'ARCH-2', 'VIST-3', 'ARCH-4', 'ARCH-6', 'COSC-4', 'LAND-6', 'CARC-4', 'LAND-1', 'ARTS-2', 'ARCH-3', 'VIST-2', 'VIST-1', 'URPN-4', 'ARTS-3', 'URPN-2', 'COSC-1', 'LAND-3', 'COSC-6', 'URPN-3', 'ARTS-1', 'VIZA-6', 'VIST-4', 'ARTS-4', 'CARC-6', 'LAND-2', 'ENDS-1'}
https://web-as.tamu.edu/GradeReports/PDFReports/20192/grd20192AR.pdf
{'PLAN-6', 'CARC-3', 'COSC-3', 'COSC-2', 'ARCH-2', 'VIST-3', 'ARCH-4', 'ARCH-6', 'COSC-4', 'LAND-6', 'ARTS-2', 'LAND-2', 'VIST-2', 'ARCH-3', 'VIST-1', 'ARTS-3', 'URPN-2', 'ARTS-1', 'VIZA-6', 'VIST-4', 'URPN-3', 'ENDS-1'}
https://web-as.tamu.edu/GradeReports/PDFReports/20193/grd20193AR.pdf
{'LDEV-6', 'LAND-4', 'PLAN-6', 'CARC-3', 'COSC-3', 'URSC-6', 'COSC-2', 'ARCH-2', 'VIST-3', 'ARCH-4', 'ARCH-6', 'COSC-4', 'LAND-6', 'CARC-4', 'LAND-1', 'ARTS-2', 'ARCH-3', 'VIST-2', 'VIST-1', 'URPN-4', 'ARTS-3', 'URPN-2', 'CARC-1', 'COSC-1', 'LAND-3', 'COSC-6', 'URPN-3', 'ARTS-1', 'VIZA-6', 'VIST-4', 'CARC-6',

https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213BA.pdf
{'BUSN-4', 'MGMT-6', 'ISTM-6', 'BUSN-2', 'MGMT-2', 'SCMT-4', 'SCMT-3', 'BUSN-1', 'ANLY-6', 'FINC-2', 'FINC-6', 'SCMT-6', 'MKTG-4', 'IBUS-4', 'ISTM-4', 'ACCT-7', 'ACCT-3', 'FINC-3', 'MGMT-7', 'BUAD-6', 'MGMT-4', 'MKTG-3', 'MGMT-3', 'ISTM-2', 'ISTM-3', 'BUAD-7', 'ACCT-4', 'MKTG-6', 'FINC-4', 'ACCT-6', 'FINC-7', 'ACCT-2'}
https://web-as.tamu.edu/GradeReports/PDFReports/20181/grd20181DN.pdf
{'PEDD-6', 'OMFS-6', 'OMFP-6', 'DDHS-3', 'DDHS-4', 'ORTH-6', 'PROS-6', 'AEGD-6', 'OBIO-6', 'OMFR-6'}
https://web-as.tamu.edu/GradeReports/PDFReports/20182/grd20182DN.pdf
{'PEDD-6', 'OMFS-6', 'DDHS-4', 'ORTH-6', 'AEGD-6', 'OBIO-6'}
https://web-as.tamu.edu/GradeReports/PDFReports/20183/grd20183DN.pdf
{'PEDD-6', 'OMFS-6', 'DDHS-3', 'DDHS-4', 'PERI-6', 'ORTH-6', 'AEGD-6', 'OBIO-6'}
https://web-as.tamu.edu/GradeReports/PDFReports/20191/grd20191DN.pdf
{'PEDD-6', 'OMFS-6', 'OMFP-6', 'DDHS-3', 'DDHS-4', 'ORTH-6', 'AEGD-6', 'OBIO-6', 'OMFR-6'}


https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212ED.pdf
{'TEED-6', 'CPSY-6', 'EPSY-4', 'DCED-3', 'EDAD-6', 'HLTH-4', 'EPSY-6', 'SPMT-2', 'HLTH-2', 'ATTR-6', 'KINE-1', 'TCMG-2', 'TCMG-3', 'KINE-3', 'DCED-2', 'HLTH-3', 'EDTC-6', 'EHRD-4', 'SPMT-3', 'EHRD-3', 'EDCI-6', 'EHRD-2', 'RDNG-3', 'TCMG-4', 'HLTH-6', 'KINE-4', 'KINE-2', 'BIED-6', 'INST-3', 'SPED-6', 'EDCI-7', 'EHRD-6', 'SPMT-4', 'INST-2', 'SPMT-6', 'SPSY-6', 'RDNG-4', 'RDNG-6'}
https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213ED.pdf
{'TEED-6', 'CPSY-6', 'ATTR-2', 'EPSY-4', 'KNFB-3', 'DCED-3', 'EDAD-6', 'TEFB-4', 'MASC-3', 'HLTH-4', 'EDCI-3', 'EPSY-6', 'SPED-3', 'HLTH-2', 'ATTR-6', 'KINE-1', 'SPMT-2', 'MASC-4', 'TCMG-2', 'CEHD-6', 'TCMG-3', 'KINE-3', 'DCED-2', 'HLTH-3', 'HEFB-3', 'EHRD-4', 'SPMT-3', 'EHRD-3', 'KNFB-4', 'EDCI-6', 'EHRD-2', 'MEFB-4', 'LDTC-6', 'SPED-4', 'RDNG-3', 'TCMG-4', 'HLTH-6', 'DCED-4', 'KINE-2', 'KINE-4', 'EPFB-2', 'INST-3', 'SPED-6', 'SEFB-4', 'EHRD-6', 'ATTR-3', 'SPMT-4', 'EDCI-7', 

https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212EN.pdf
{'AERO-2', 'CSCE-7', 'MSEN-3', 'MEEN-3', 'MMET-2', 'MEEN-6', 'ENGR-6', 'ESET-4', 'ESET-3', 'CSCE-4', 'ISEN-4', 'IDIS-3', 'CHEN-3', 'CSCE-1', 'MEEN-2', 'CSCE-2', 'PETE-6', 'MMET-4', 'ENGR-2', 'CHEN-4', 'ENGR-1', 'MEEN-4', 'BMEN-4', 'PETE-4', 'CVEN-3', 'MMET-3', 'CVEN-6', 'ECEN-2', 'IDIS-4', 'ICPE-6', 'BMEN-3', 'ENGR-4', 'ENGR-3', 'BMEN-2', 'ISEN-2', 'ECEN-3', 'ITDE-3', 'NUEN-3', 'CSCE-3', 'ESET-2', 'MMET-1', 'MSEN-2', 'CHEN-2', 'AERO-3', 'PETE-3', 'ECEN-4', 'CVEN-4', 'ISEN-6', 'OCEN-6', 'EVEN-3', 'MSEN-6', 'OCEN-3', 'ISEN-3'}
https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213EN.pdf
{'ITDE-4', 'AERO-2', 'IDIS-2', 'ESET-3', 'PETE-6', 'BIOT-6', 'ENGR-2', 'NUEN-1', 'IDIS-6', 'CVEN-6', 'IDIS-4', 'ENGR-4', 'SENG-3', 'ESET-2', 'OCEN-4', 'OCEN-6', 'EVEN-3', 'OCEN-3', 'MTDE-4', 'MSEN-3', 'MMET-2', 'ENGR-6', 'ISEN-4', 'AREN-4', 'MEEN-2', 'PETE-4', 'BMEN-1', 'CVEN-3', 'CLEN-1', 'CSCE-6', 'ICPE-6', 'AERO-6', 'BMEN-3', 

https://web-as.tamu.edu/GradeReports/PDFReports/20192/grd20192LA.pdf
{'SPAN-1', 'COMM-2', 'HIST-2', 'ECON-2', 'ANTH-2', 'ARAB-4', 'ECON-3', 'SOCI-2', 'JAPN-2', 'THAR-2', 'POLS-3', 'ENGL-2', 'WGST-3', 'ARAB-3', 'SPAN-2', 'HIST-1', 'MUSC-2', 'ANTH-3', 'ECON-4', 'PSYC-1', 'MUSC-3', 'GERM-2', 'RUSS-4', 'SOCI-4', 'PSYC-4', 'PERF-2', 'SPAN-4', 'PHIL-2', 'PSYC-3', 'SOCI-3', 'COMM-3', 'FILM-3', 'ENGL-3', 'ARAB-2', 'SPAN-3', 'COMM-4', 'POLS-2', 'GERM-3', 'HIST-3', 'POLS-4', 'JAPN-4', 'PSYC-6', 'JAPN-3', 'PHIL-1'}
https://web-as.tamu.edu/GradeReports/PDFReports/20193/grd20193LA.pdf
{'THAR-3', 'RELS-2', 'ANTH-2', 'CLAS-3', 'WGST-3', 'HIST-1', 'ANTH-3', 'ENGL-4', 'LING-3', 'SOCI-6', 'MUSC-3', 'RUSS-4', 'FREN-3', 'LMAS-2', 'ITAL-2', 'LBAR-1', 'ECMT-6', 'SOCI-3', 'PERF-1', 'COMM-3', 'ECMT-4', 'INTS-2', 'FILM-2', 'HIST-4', 'JAPN-4', 'PSYC-6', 'CLAS-4', 'JOUR-4', 'JAPN-3', 'PHIL-1', 'FREN-1', 'ARAB-4', 'ECON-3', 'ARAB-1', 'POLS-3', 'THAR-4', 'SPAN-2', 'PERF-3', 'MUSC-2', 'ECON-4', 'PSYC-1', 'RELS-4', 

https://web-as.tamu.edu/GradeReports/PDFReports/20192/grd20192MS.pdf
set()
https://web-as.tamu.edu/GradeReports/PDFReports/20193/grd20193MS.pdf
{'SOMS-2', 'NVSC-2', 'AERS-4', 'MLSC-2', 'MLSC-4', 'NVSC-3', 'SOMS-1', 'AERS-2', 'MLSC-1', 'NVSC-4', 'MLSC-3', 'SOMS-3', 'SOMS-4', 'NVSC-1', 'AERS-1', 'AERS-3'}
https://web-as.tamu.edu/GradeReports/PDFReports/20201/grd20201MS.pdf
{'SOMS-2', 'NVSC-2', 'AERS-4', 'MLSC-2', 'MLSC-4', 'NVSC-3', 'AERS-2', 'SOMS-1', 'MLSC-1', 'NVSC-4', 'MLSC-3', 'SOMS-3', 'SOMS-4', 'AERS-1', 'AERS-3'}
https://web-as.tamu.edu/GradeReports/PDFReports/20202/grd20202MS.pdf
set()
https://web-as.tamu.edu/GradeReports/PDFReports/20203/grd20203MS.pdf
{'SOMS-2', 'NVSC-2', 'AERS-4', 'MLSC-2', 'MLSC-4', 'NVSC-3', 'SOMS-1', 'AERS-2', 'MLSC-1', 'NVSC-4', 'MLSC-3', 'SOMS-3', 'SOMS-4', 'NVSC-1', 'AERS-1', 'AERS-3'}
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211MS.pdf
{'SOMS-2', 'NVSC-2', 'AERS-4', 'MLSC-2', 'MLSC-4', 'NVSC-3', 'SOMS-1', 'AERS-2', 'MLSC-1', 'NVSC-4', 

https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213SC.pdf
{'PHYS-1', 'NRSC-6', 'SCEN-2', 'PHYS-2', 'CHEM-6', 'PHYS-6', 'STAT-4', 'CHEM-1', 'STAT-3', 'ASTR-6', 'ASTR-1', 'PHYS-4', 'PHYS-3', 'CHEM-4', 'BIOL-4', 'MATH-3', 'BIOL-2', 'SCEN-1', 'MATH-4', 'NRSC-3', 'NRSC-2', 'MATH-1', 'BIOL-1', 'EEBL-6', 'BIOL-6', 'MATH-6', 'NRSC-4', 'STAT-2', 'CHEM-2', 'MATH-2', 'CHEM-3', 'ASTR-3', 'STAT-6', 'ASTR-4', 'BIOL-3', 'STAT-1'}
https://web-as.tamu.edu/GradeReports/PDFReports/20181/grd20181VM.pdf
{'VTPB-4', 'BIMS-3', 'VMID-6', 'VTPP-1', 'VTPB-2', 'VTPP-2', 'BIMS-1', 'VIBS-2', 'BIMS-2', 'VTPP-6', 'VTMI-6', 'VPAR-6', 'VTPP-3', 'VTPB-3', 'VPAT-6', 'BIMS-4', 'VIBS-4', 'VIBS-6', 'VLCS-4', 'VTPP-4', 'VIBS-3'}
https://web-as.tamu.edu/GradeReports/PDFReports/20182/grd20182VM.pdf
{'VIBS-3', 'VTPB-4', 'VTPP-4', 'VIBS-6'}
https://web-as.tamu.edu/GradeReports/PDFReports/20183/grd20183VM.pdf
{'VIBS-4', 'BIMS-3', 'VTPP-2', 'VTPB-4', 'VTMI-6', 'VTPP-3', 'VTPB-3', 'VMID-6', 'BIMS-1', 'VIBS-2', 'VIBS-6', 

https://web-as.tamu.edu/GradeReports/PDFReports/20221/grd20221NU.pdf
{'NURS-3', 'NURS-4', 'NURS-6'}
https://web-as.tamu.edu/GradeReports/PDFReports/20221/grd20221PH.pdf
{'HPCH-6', 'PHLT-3', 'PHEB-6', 'PHLT-2', 'SOPH-6', 'PHPM-6', 'PHEO-6', 'PHLT-4'}
https://web-as.tamu.edu/GradeReports/PDFReports/20221/grd20221SC.pdf
{'PHYS-1', 'NRSC-6', 'SCEN-2', 'PHYS-2', 'CHEM-6', 'PHYS-6', 'STAT-4', 'CHEM-1', 'STAT-3', 'ASTR-6', 'ASTR-1', 'PHYS-4', 'PHYS-3', 'CHEM-4', 'BIOL-4', 'MATH-3', 'BIOL-2', 'SCEN-1', 'MATH-4', 'NRSC-2', 'MATH-1', 'BIOL-1', 'EEBL-6', 'BIOL-6', 'MATH-6', 'NRSC-4', 'STAT-2', 'CHEM-2', 'MATH-2', 'CHEM-3', 'ASTR-3', 'STAT-6', 'ASTR-4', 'BIOL-3'}
https://web-as.tamu.edu/GradeReports/PDFReports/20221/grd20221VM.pdf
{'VTPB-4', 'BIMS-3', 'VMID-6', 'VTPP-1', 'VTPB-2', 'VTPP-2', 'BIMS-1', 'VIBS-2', 'BIMS-2', 'VTPP-6', 'VTMI-6', 'VTPP-3', 'VTPB-3', 'BIMS-4', 'VIBS-4', 'VIBS-1', 'VIBS-6', 'VLCS-4', 'VTPP-4', 'VIBS-3'}


In [43]:
col_names = ['Course_Info', 'A', 'A percentage', 'B', 'B percentage',  'C', 'C percentage',
             'D', 'D percentage', 'F', 'F percentage', 'Total Completed', 'GPA',
             'I','S','U','Q','X', 'Total Registered', 'Instructor', 'Year','Term','College']
df = pd.DataFrame(rows, columns = col_names)

In [44]:
df.head()

,Course_Info,A,A percentage,B,B percentage,C,C percentage,D,D percentage,F,...,I,S,U,Q,X,Total Registered,Instructor,Year,Term,College
0,AGCJ-105-501,55,87.30%,7,11.11%,1,1.59%,0,0.00%,0,...,0,0,0,0,0,63,WALTHER D,2017,3,AG
1,AGCJ-105-502,16,80.00%,4,20.00%,0,0.00%,0,0.00%,0,...,0,0,0,0,0,20,WALTHER D,2017,3,AG
2,AGCJ-281-500,16,51.61%,11,35.48%,4,12.90%,0,0.00%,0,...,0,0,0,1,0,32,LEGGETTE H,2017,3,AG
3,AGCJ-305-599,27,50.00%,20,37.04%,5,9.26%,1,1.85%,1,...,0,0,0,0,0,54,DUNSFORD D,2017,3,AG
4,AGCJ-305-700,12,50.00%,4,16.67%,5,20.83%,2,8.33%,1,...,0,0,0,0,0,24,DUNSFORD D,2017,3,AG


In [56]:
#Divide up Course Info for more convenient analysis in R
df['Course'] =  df['Course_Info'].str[:8]
df['Course_Num'] =  df['Course_Info'].str[5:8]
df['Dept'] = df['Course_Info'].str[:4]
df['Section'] = df['Course_Info'].str[9:12]
df['TermYear'] = df['Year'] + ' ' + df['Term']

In [63]:
conditions = [
    (df['Term'] == '1'),
    (df['Term'] == '2'),
    (df['Term'] == '3')    
]
choices = ['Spring', 'Summer', 'Fall']
df['Semester'] = np.select(conditions, choices)

In [65]:
df.to_csv('tamu_grade_reports.csv', index = False)